<a href="https://colab.research.google.com/github/vladimirrim/QA_DL/blob/develop/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_pretrained_bert
!pip install transformers

In [2]:
import torch
import numpy as np
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from transformers import  BertModel
from pytorch_pretrained_bert import BertConfig, BertForPreTraining

In [0]:
from torch import nn
from torch.nn import CrossEntropyLoss

In [0]:
import torch.nn.functional as F

class BertForQuestionAnswering(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.bert.eval()
        self.qa_outputs = nn.Sequential(nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(64, 2))
        self.loss_fct = CrossEntropyLoss()
        
    

    def forward(self, input_ids=None, token_type_ids=None, start_positions=None, end_positions=None, mask=None):
        output = self.bert(input_ids, attention_mask=mask)

        sequence_output = output[0]

        logits = self.qa_outputs(sequence_output)
        loss = None

        if start_positions is not None and end_positions is not None:
            loss = (self.loss_fct(logits[:, :, 0].masked_fill((1 - mask).bool(), float('-inf')), start_positions) + \
                   self.loss_fct(logits[:, :, 1].masked_fill((1 - mask).bool(), float('-inf')), end_positions)) / 2

        return loss, F.softmax(logits.masked_fill((1 - mask[:, :, None]).bool(), float('-inf')), dim=1)

In [0]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [0]:
def preprocess(text, question, answer):
    answer = answer.lower()
    if answer not in text.lower():
        return [], []
    
    firstInText = text.lower().find(answer)
    lastInText = firstInText + len(answer)
    text_tokens = tokenizer.tokenize(text[:firstInText].strip())
    first = len(text_tokens)
    text_tokens += tokenizer.tokenize(text[firstInText:lastInText].strip())
    last = len(text_tokens) - 1
    text_tokens += tokenizer.tokenize(text[lastInText:].strip())
    question_tokens = tokenizer.tokenize(question)
    
    length = MAX_TEXT_LEN - len(question_tokens) - 3
    if len(text_tokens) > length:
        part_length = length // 3
        stride = 3 * part_length
        nrow = np.ceil(len(text_tokens) / part_length) - 2
        indexes = part_length * np.arange(nrow)[:, None] + np.arange(stride)
        indexes = indexes.astype(np.int32)

        max_index = indexes.max()
        diff = max_index + 1 - len(text_tokens)
        text_tokens += diff * [tokenizer.pad_token]

        text_tokens = list(np.array(text_tokens)[indexes])
        
        tokens = []
        labels = []
        for i, ts in enumerate(text_tokens):
            while ts[-1] == tokenizer.pad_token:
                ts = ts[:-1]
                
            tokens += [ts]
                
            lfirst = first - i * part_length
            llast = last - i * part_length
            
            mask = lfirst >= 0 and lfirst < len(ts) and llast >= 0 and llast < len(ts)
            labels += [(lfirst if mask else 0, llast if mask else 0)]
    else:
        tokens = [text_tokens]
        labels = [(first, last)]
        
    for i in range(len(tokens)):
        # TODO удалять этот костыль!!!
        if str(type(tokens[i])) == "<class 'numpy.ndarray'>": 
            tokens[i] = list(tokens[i])
        tokens[i] = [tokenizer.cls_token] + \
                    question_tokens + \
                    [tokenizer.sep_token] + \
                    tokens[i] + \
                    [tokenizer.sep_token]
        labels[i] = (labels[i][0] + 2 + len(question_tokens), labels[i][1] + 2 + len(question_tokens))

    return tokens, labels

In [0]:
def pad_sequence(texts):
    max_len = max([len(text) for text in texts])
    masks = [[1] * len(text) + [0] * (max_len - len(text)) for text in texts]
    texts = [text + [tokenizer.pad_token] * (max_len - len(text)) for text in texts]
    texts = [tokenizer.convert_tokens_to_ids(text) for text in texts]
    texts = torch.LongTensor(texts)
    masks = torch.LongTensor(masks)

    return texts, masks

def collate_fn(data):
    texts, labels = zip(*data)
    texts, masks = pad_sequence(texts)
    
    labels_first, labels_last = zip(*labels)
    start_pos = labels_first
    end_pos = labels_last
    return texts, masks, torch.LongTensor(start_pos), torch.LongTensor(end_pos)

In [0]:
MAX_TEXT_LEN = 256

In [9]:
from google.colab import drive
import json

drive.mount('./gdrive')
train_dataset = './gdrive/My Drive/datasets_for_homeworks/train-v1.1.json'
dev_dataset = './gdrive/My Drive/datasets_for_homeworks/dev-v1.1.json'
with open(train_dataset, 'r') as train_json, open(dev_dataset, 'r') as dev_json:
    train_data = json.load(train_json)
    dev_data = json.load(dev_json)

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [0]:
def get_text_question_ans_dataset(squad_dataset):
    tqa_dataset = []
    for d in squad_dataset['data']:
        for p in d['paragraphs']:
            for qa in p['qas']:
                # TODO: deal with several answers
                tqa_dataset.append((p['context'], qa['question'], qa['answers'][0]['text']))
    return tqa_dataset

In [0]:
tqa_train_dataset = get_text_question_ans_dataset(train_data)
tqa_dev_dataset = get_text_question_ans_dataset(dev_data)

In [12]:
print(len(tqa_train_dataset))
print(len(tqa_dev_dataset))
print(f'Max text len in train: {max(map(lambda x: len(x[0]), tqa_train_dataset))}')
print(f'Max text len in dev: {max(map(lambda x: len(x[0]), tqa_dev_dataset))}')
print(tqa_train_dataset[0])
print(tqa_train_dataset[-1])
print(tqa_dev_dataset[0])
print(tqa_dev_dataset[-1])

87599
10570
Max text len in train: 3706
Max text len in dev: 4063
('Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'Saint Bernadette Soubirous')
("Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secr

In [13]:
from tqdm.auto import tqdm

dataset_tokens, dataset_labels = [], []
for datapoint in tqdm(tqa_train_dataset):
    tokens, labels = preprocess(datapoint[0], datapoint[1], datapoint[2])
    dataset_tokens += tokens
    dataset_labels += labels

KeyboardInterrupt: ignored

In [0]:
print(len(dataset_tokens))

In [0]:
print(dataset_tokens[0])
print(dataset_tokens[-1])

In [0]:
train_data_loader = torch.utils.data.DataLoader(list(zip(dataset_tokens, dataset_labels)), batch_size=16, shuffle=True,collate_fn=collate_fn)

In [0]:
!pip3 install wandb

In [0]:
!wandb login

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 2377ef66e63c2eda02e1d83797d0cc73170988c7
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import wandb
wandb.init(project="dul")

W&B Run: https://app.wandb.ai/ram_saw/dul/runs/r85f4nnw

In [14]:
model = BertForQuestionAnswering()
model.load_state_dict(torch.load('./gdrive/My Drive/bert.pt'))

<All keys matched successfully>

In [21]:
device = 'cuda'
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [0]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 0.00005, weight_decay=0.000001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)
epochs = 3

for epoch in range(epochs):
    model.train()
    for i, (texts, masks, start_pos, end_pos) in enumerate(train_data_loader):
        optimizer.zero_grad()
        loss, _ = model(texts.to(device),
                        mask=masks.to(device),
                        start_positions=torch.tensor(start_pos).to(device),
                        end_positions=torch.tensor(end_pos).to(device))
        wandb.log({'loss' : float(loss)})
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Model saved on {i} iteration!')
            torch.save(model.state_dict(), './gdrive/My Drive/bert.pt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Model saved on 0 iteration!
Model saved on 100 iteration!
Model saved on 200 iteration!
Model saved on 300 iteration!
Model saved on 400 iteration!
Model saved on 500 iteration!
Model saved on 600 iteration!
Model saved on 700 iteration!
Model saved on 800 iteration!
Model saved on 900 iteration!
Model saved on 1000 iteration!
Model saved on 1100 iteration!
Model saved on 1200 iteration!
Model saved on 1300 iteration!
Model saved on 1400 iteration!
Model saved on 1500 iteration!
Model saved on 1600 iteration!
Model saved on 1700 iteration!
Model saved on 1800 iteration!
Model saved on 1900 iteration!
Model saved on 2000 iteration!
Model saved on 2100 iteration!
Model saved on 2200 iteration!
Model saved on 2300 iteration!
Model saved on 2400 iteration!
Model saved on 2500 iteration!
Model saved on 2600 iteration!
Model saved on 2700 iteration!


In [15]:
model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [0]:
import torch.nn.functional as F
import re

def getBestProb(probs):
    n = len(probs)
    start, end, bestProb = 0, 0, 0
    for i in range(n):
        for j in range(i, n):
            prob = probs[i, 0] * probs[j, 1]
            if bestProb < prob:
                bestProb, start, end = prob, i, j
              
    return start, end


def concat(tokens):
    tokens = [token.replace('#', '') for token in tokens]
    return ' '.join(list(filter(lambda s: s != tokenizer.unk_token, tokens))).strip()

In [17]:
from tqdm.auto import tqdm

dev_dataset_tokens, dev_dataset_labels = [], []
for datapoint in tqdm(tqa_dev_dataset):
    tokens, labels = preprocess(datapoint[0], datapoint[1], datapoint[2])
    dev_dataset_tokens += tokens
    dev_dataset_labels += labels

In [0]:
dev_data_loader = torch.utils.data.DataLoader(list(zip(dev_dataset_tokens, dev_dataset_labels)), batch_size=16, shuffle=True,collate_fn=collate_fn)

In [0]:
def cov(m, rowvar=True, inplace=False):
    '''Estimate a covariance matrix given data.

    Covariance indicates the level to which two variables vary together.
    If we examine N-dimensional samples, `X = [x_1, x_2, ... x_N]^T`,
    then the covariance matrix element `C_{ij}` is the covariance of
    `x_i` and `x_j`. The element `C_{ii}` is the variance of `x_i`.

    Args:
        m: A 1-D or 2-D array containing multiple variables and observations.
            Each row of `m` represents a variable, and each column a single
            observation of all those variables.
        rowvar: If `rowvar` is True, then each row represents a
            variable, with observations in the columns. Otherwise, the
            relationship is transposed: each column represents a variable,
            while the rows contain observations.

    Returns:
        The covariance matrix of the variables.
    '''
    if m.dim() > 2:
        raise ValueError('m has more than 2 dimensions')
    if m.dim() < 2:
        m = m.view(1, -1)
    if not rowvar and m.size(0) != 1:
        m = m.t()
    # m = m.type(torch.double)  # uncomment this line if desired
    fact = 1.0 / (m.size(1) - 1)
    if inplace:
        m -= torch.mean(m, dim=1, keepdim=True)
    else:
        m = m - torch.mean(m, dim=1, keepdim=True)
    mt = m.t()  # if complex: mt = m.t().conj()
    return fact * m.matmul(mt).squeeze()

In [0]:
def test_model(model):
  model.to(device)
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for i, (texts, masks, start_pos, end_pos) in enumerate(dev_data_loader):
        _, probs = model(texts.to(device),
                        mask=masks.to(device),
                        start_positions=torch.tensor(start_pos).to(device),
                        end_positions=torch.tensor(end_pos).to(device))
        starts, ends = [], []
        for prob in probs:
            start_b, end_b = getBestProb(prob)
            starts.append(start_b)
            ends.append(end_b)
        starts = torch.tensor(starts)
        ends = torch.tensor(ends)
        print(starts == torch.tensor(start_pos))
        print(ends == torch.tensor(end_pos))
        print((starts == torch.tensor(start_pos)) * (ends == torch.tensor(end_pos)))
        print(torch.sum((starts == torch.tensor(start_pos)) * (ends == torch.tensor(end_pos))))
        correct += torch.sum((starts == torch.tensor(start_pos)) * (ends == torch.tensor(end_pos)))
        total += len(starts)
        if i % 1 == 0:
            print(f'{i} iteration')
            print(f'Accuracy on dev data is {correct / total}')
  print(f'Accuracy on dev data is {correct / total}')

In [2]:
test_model(model)

NameError: ignored